# Master Discogs Database

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap

### Master DB code
from masterdb import masterdb
from mainDB import mainDB


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Notebook Last Run Initiated: 2020-08-29 19:23:07.971155


In [2]:
mdb = myMusicDBMap(debug=True)

Creating myMusicDBMap()
   Loading my music db map: /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/myMusicMap.p
   DB keys: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']
Found 4206 artist entries
Counter({'Discogs': 4101, 'AllMusic': 3991, 'MusicBrainz': 3846, 'LastFM': 3228, 'CDandLP': 755, 'DatPiff': 545, 'RockCorner': 491, 'AceBootlegs': 178, 'RateYourMusic': 68, 'MusicStack': 5, 'MetalStorm': 0})
Counter({'Discogs': 4101, 'AllMusic': 3991, 'MusicBrainz': 3846, 'LastFM': 3228, 'CDandLP': 755, 'DatPiff': 545, 'RockCorner': 491, 'AceBootlegs': 178, 'RateYourMusic': 68, 'MusicStack': 5, 'MetalStorm': 0})


In [3]:
maindb = mainDB(mdb=mdb, create=True, debug=True)
print("\n\n{0}\n".format("Full DB"))
maindb.setDBFull()
print("\n\n{0}\n".format("Known DB"))
#maindb.setDBKnown() ## Do this to recreate everything
#maindb.setDBKnown()
#artistDBs = maindb.getKnownArtistDBs()

Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   datpiff   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================
  Creating Database Records for Discogs
  Creating Database Records for AllMusic
  Creating Database Records for MusicBrainz
  Creating Database Records for LastFM
  Creating Database Records for RockCorner
  Creating Database Records for DatPiff
  Creating Database Records for AceBootlegs
  Creating Databas

/Volumes/Piggy/Discog/artists-discogs-db/metadata/91-Metadata.p  	91 691615
/Volumes/Piggy/Discog/artists-discogs-db/metadata/92-Metadata.p  	92 699296
/Volumes/Piggy/Discog/artists-discogs-db/metadata/93-Metadata.p  	93 706807
/Volumes/Piggy/Discog/artists-discogs-db/metadata/94-Metadata.p  	94 714323
/Volumes/Piggy/Discog/artists-discogs-db/metadata/95-Metadata.p  	95 721901
/Volumes/Piggy/Discog/artists-discogs-db/metadata/96-Metadata.p  	96 729342
/Volumes/Piggy/Discog/artists-discogs-db/metadata/97-Metadata.p  	97 736854
/Volumes/Piggy/Discog/artists-discogs-db/metadata/98-Metadata.p  	98 744305
/Volumes/Piggy/Discog/artists-discogs-db/metadata/99-Metadata.p  	99 751847



Saving 751847 entries to /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToRef.p

Saving data to /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToRef.p
  --> This file is 16.8MB.
Saved data to /Users/tgadfort/Music/Discog/db-discogs/ArtistIDToRef.p
  --> This file is 16.8MB.
Saving 751847 entries to /Users/tgad

/Volumes/Piggy/Discog/artists-discogs-db/metadata/59-MediaMetadata.p 	420272    916115    3934031   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/6-MediaMetadata.p 	427715    932252    4003532   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/60-MediaMetadata.p 	435242    949184    4078643   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/61-MediaMetadata.p 	442727    966592    4151175   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/62-MediaMetadata.p 	450325    983278    4219057   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/63-MediaMetadata.p 	457915    999043    4290063   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/64-MediaMetadata.p 	465561    1015420   4359656   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/65-MediaMetadata.p 	473131    1032922   4434617   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/66-MediaMetadata.p 	480661    1049522   4505318   
/Volumes/Piggy/Discog/artists-discogs-db/metadata/67-MediaMetadata.p 	488272    1065331   45

/Volumes/Piggy/Discog/artists-allmusic-db/metadata/12-Metadata.p  	23 12737
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/20-Metadata.p  	24 13275
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/27-Metadata.p  	25 13817
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/6-Metadata.p  	26 14345
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/13-Metadata.p  	27 14876
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/28-Metadata.p  	28 15421
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/42-Metadata.p  	29 15926
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/35-Metadata.p  	30 16450
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/49-Metadata.p  	31 17025
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/43-Metadata.p  	32 17551
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/29-Metadata.p  	33 18077
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/36-Metadata.p  	34 18588
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/50-Metadata.p  	35 19182
/Volumes/Pigg

/Volumes/Piggy/Discog/artists-allmusic-db/metadata/15-MediaMetadata.p 	2629      10917     30379     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/22-MediaMetadata.p 	3142      13206     36333     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/1-MediaMetadata.p 	3638      15329     42186     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/8-MediaMetadata.p 	4180      17674     47991     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/16-MediaMetadata.p 	4715      19978     55098     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/23-MediaMetadata.p 	5290      22598     62374     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/2-MediaMetadata.p 	5838      24625     68812     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/9-MediaMetadata.p 	6378      27232     75569     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/24-MediaMetadata.p 	6925      29493     81616     
/Volumes/Piggy/Discog/artists-allmusic-db/metadata/17-MediaMetadata.p 	7467      32175

...

Current Time is Sat Aug 29, 2020 19:39:54 for Creating Artist DBs
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/0-MediaMetadata.p 	64        409       409       
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/1-MediaMetadata.p 	128       866       866       
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/10-MediaMetadata.p 	189       1349      1349      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/11-MediaMetadata.p 	234       1735      1735      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/12-MediaMetadata.p 	276       2054      2054      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/13-MediaMetadata.p 	328       2457      2457      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/14-MediaMetadata.p 	363       2768      2768      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/15-MediaMetadata.p 	408       3009      3009      
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/16-MediaMetadata.p 	454       3355      3355      
/Volumes/Piggy/Discog/artis

/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/95-MediaMetadata.p 	4771      38057     38057     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/96-MediaMetadata.p 	4819      38406     38406     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/97-MediaMetadata.p 	4871      38726     38726     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/98-MediaMetadata.p 	4923      39162     39162     
/Volumes/Piggy/Discog/artists-cdandlp-db/metadata/99-MediaMetadata.p 	4977      39600     39600     



Saving 4977 entries to /Users/tgadfort/Music/Discog/db-cdandlp/ArtistIDToAlbumNames.p

Saving data to /Users/tgadfort/Music/Discog/db-cdandlp/ArtistIDToAlbumNames.p
  --> This file is 1.3MB.
Saved data to /Users/tgadfort/Music/Discog/db-cdandlp/ArtistIDToAlbumNames.p
  --> This file is 1.3MB.
Saving 4977 entries to /Users/tgadfort/Music/Discog/db-cdandlp/ArtistIDToAlbumRefs.p

Saving data to /Users/tgadfort/Music/Discog/db-cdandlp/ArtistIDToAlbumRefs.p
  --> This file is 1.6MB.
Saved dat

/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/10-MediaMetadata.p 	4         0         254       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/16-MediaMetadata.p 	6         0         354       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/18-MediaMetadata.p 	8         0         478       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/19-MediaMetadata.p 	9         0         484       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/20-MediaMetadata.p 	10        0         486       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/21-MediaMetadata.p 	11        0         566       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/22-MediaMetadata.p 	12        0         589       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/23-MediaMetadata.p 	13        0         619       
/Volumes/Piggy/Discog/artists-rateyourmusic-db/metadata/26-MediaMetadata.p 	14        0         657       
/Volumes/Piggy/Discog/artists-rateyou

/Volumes/Piggy/Discog/artists-musicstack-db/metadata/0-Metadata.p  	0 2
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/1-Metadata.p  	1 3
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/10-Metadata.p  	2 7
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/11-Metadata.p  	3 12
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/12-Metadata.p  	4 13
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/13-Metadata.p  	5 16
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/16-Metadata.p  	6 17
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/17-Metadata.p  	7 18
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/18-Metadata.p  	8 21
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/19-Metadata.p  	9 23
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/2-Metadata.p  	10 26
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/20-Metadata.p  	11 27
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/21-Metadata.p  	12 29
/Volumes/Piggy/Discog/artists-musicstack-

/Volumes/Piggy/Discog/artists-musicstack-db/metadata/10-MediaMetadata.p 	7         0         8         
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/11-MediaMetadata.p 	12        0         25        
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/12-MediaMetadata.p 	13        0         27        
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/13-MediaMetadata.p 	16        0         31        
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/16-MediaMetadata.p 	17        0         32        
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/17-MediaMetadata.p 	18        0         33        
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/18-MediaMetadata.p 	21        0         37        
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/19-MediaMetadata.p 	23        0         49        
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/2-MediaMetadata.p 	26        0         67        
/Volumes/Piggy/Discog/artists-musicstack-db/metadata/20-MediaMeta

In [ ]:
maindb = mainDB(mdb=mdb, create=False, debug=True)

In [ ]:
maindb.setDBFull()

# Metadata

In [ ]:
%load_ext autoreload
%autoreload

from multiprocessing import Pool
from mainDB import mainDB
import time
mainDB = mainDB()
dbdata = mainDB.dbdata

def parseArtistsAM(modVal, force=False, doExtra=False):
    dbdata["AllMusic"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsAM.parseArtistModValFiles(modVal, force=force)

def parseArtistsDC(modVal, force=True, doExtra=False):
    dbdata["Discogs"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsDC.parseArtistModValFiles(modVal, force=force)

def parseArtistsMB(modVal, force=False, doExtra=False):
    dbdata["MusicBrainz"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsMB.parseArtistModValFiles(modVal, force=force)

def parseArtistsAB(modVal, force=False, doExtra=False):
    artsAB.parseArtistFiles(force=force)
    
def parseArtistsDP(modVal, force=False, doExtra=False):
    dbdata['DatPiff']['Artists'].parseArtistFiles()
    #artsDP.parseArtistFiles(force=force)

def parseArtistsRM(modVal, force=False, doExtra=False):
    dbdata["RateYourMusic"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsRM.parseArtistModValFiles(modVal, force=force)

def parseArtistsLM(modVal, force=False, doExtra=False):
    dbdata["LastFM"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsLM.parseArtistModValFiles(modVal, force=force)

def parseArtistsRC(modVal, force=False, doExtra=False):
    dbdata["RockCorner"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsRC.parseArtistModValFiles(modVal, force=force)

def parseArtistsCL(modVal, force=False, doExtra=False):
    dbdata["CDandLP"]["Artists"].parseArtistModValFiles(modVal, force=force)
    #artsCL.parseArtistModValFiles(modVal, force=force)

def parseArtistsMS(modVal, force=False, doExtra=False):
    artsMS.parseArtistFiles(force=force)

def parseArtistsMT(modVal, force=False, doExtra=False):
    artsMT.parseArtistModValFiles(modVal, force=force)

    
def parseArtistsParallel(db, nProcs=3, force=False):
    pool = Pool(processes=nProcs)
    if db == "Discogs":
        result = pool.map_async(parseArtistsDC, range(100))
    elif db == "AllMusic":
        result = pool.map_async(parseArtistsAM, range(100))
    elif db == "MusicBrainz":
        result = pool.map_async(parseArtistsMB, range(100))
    elif db == "AceBootlegs":
        result = pool.map_async(parseArtistsAB, [None])
    elif db == "DatPiff":
        result = pool.map_async(parseArtistsDP, [None])
    elif db == "RateYourMusic":
        dbdata["RateYourMusic"]["Artists"].parseDownloadedFiles()
        result = pool.map_async(parseArtistsRM, range(100))
    elif db == "LastFM":
        result = pool.map_async(parseArtistsLM, range(100))
    elif db == "RockCorner":
        result = pool.map_async(parseArtistsRC, range(100))
    elif db == "CDandLP":
        result = pool.map_async(parseArtistsCL, range(100))
        #result = pool.map_async(parseArtistsCL, range(56,72))
        #result = pool.map_async(parseArtistsCL, [55,25,26])
    elif db == "MusicStack":
        result = pool.map_async(parseArtistsMS, [None])
    elif db == "MetalStorm":
        result = pool.map_async(parseArtistsMT, range(100))
    else:
        raise ValueError("[{0}] is not recognized as a DB".format(db))

    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

In [ ]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel(db="AllMusic", nProcs=3)
#parseArtistsParallel(db="MusicBrainz", nProcs=3)
parseArtistsParallel(db="Discogs", nProcs=4)
#parseArtistsParallel(db="AceBootlegs", nProcs=1)
#parseArtistsParallel(db="DatPiff", nProcs=1)
#parseArtistsParallel(db="RateYourMusic", nProcs=3)
#parseArtistsParallel(db="LastFM", nProcs=3)
#parseArtistsParallel(db="RockCorner", nProcs=3)
#parseArtistsParallel(db="CDandLP", nProcs=3)
#parseArtistsParallel(db="MusicStack", nProcs=1)
elapsed(start2, cmt2)
elapsed(start, cmt)

# Download Stuff

In [ ]:
mdb = myMusicDBMap(debug=True)
maindb = mainDB(mdb=mdb, create=False, debug=True)
dbdata = maindb.dbdata

In [ ]:
ifile = "unmatchedArtists4.p"
noMatches = getFile(ifile)
print(len(noMatches))
dbs = ['Discogs', 'AllMusic', 'MusicBrainz', 'LastFM']
dbs = ['LastFM']

#noMatches=['Go-A', 'PM', 'Qango']

from time import sleep
maxI = -1
for i,artist in enumerate(noMatches):
    print("="*100)
    print("="*100)
    print("="*100)
    print("="*100)
    print(i,'/',len(noMatches),'\t',artist)
    if i <= maxI:
        continue
    for db in dbs:
        print("="*100)
        print(i,'/',len(noMatches),'\t',artist,'===>',db)
        try:
            dbdata[db]["Artists"].searchForArtist(artist)
        except:
            sleep(5)